In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline
!git clone https://github.com/JATAYU000/02Hero

In [ ]:
words = open('/kaggle/working/02Hero/names.txt', 'r').read().splitlines()
words[:8]

In [ ]:
chars = sorted(list(set(''.join(words))))
str2int = {s:i+1 for i,s in enumerate(chars)}
str2int['.'] = 0
int2str = {i:s for s,i in str2int.items()}
print(int2str)

In [ ]:
block = 3
X,Y = [] , []

for i in words[:5]:
    print(i)
    context = [0] * block
    for ch in i +'.':
        ix = str2int[ch]
        X.append(context)
        Y.append(ix)
        print(context,' => ',int2str[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)
        

In [ ]:
# look up table cranking it down to 2 dim for now
L = torch.randn((27,2))

In [ ]:
# identical to L[5]
F.one_hot(torch.tensor(5),num_classes=27).float() @ L

In [ ]:
# L[Xi]
L[[5,6,7,7,7]]

In [ ]:
X.shape

In [ ]:
embedding = L[X]
embedding.shape

In [ ]:
w1 = torch.randn(6,100)  # 6 inputs and 100 neurons in the first layer
b1 = torch.randn(100)  # biases for the 100 neurons


In [ ]:
# embedding[:, 0, :] , embedding[:, 1, :] , embedding[:, 2, :] these are the 3 inputs 
# these are 32 x 2 i want to catenate not 0th but axis 1
torch.cat([embedding[:, 0, :] , embedding[:, 1, :] , embedding[:, 2, :]],1).shape

In [ ]:
# unbind give the list [embedding[:, 0, :] , embedding[:, 1, :] , embedding[:, 2, :]] since the block size might not be 3 all the time
torch.cat(torch.unbind(embedding,1),1).shape

In [ ]:
# efficient way in torch
embedding.view(32,6)

In [ ]:
layer1 = torch.tanh(embedding.view(embedding.shape[0],6) @ w1 + b1)
layer1.shape

In [ ]:
w2 = torch.randn(100,27)  # 100 inputs and 27 neurons in the first layer
b2 = torch.randn(27)  # biases for the 27 output neurons

In [ ]:
logits = layer1 @ w2 + b2
logits.shape

In [ ]:
fake_counts = logits.exp()
p = fake_counts / fake_counts.sum(1, keepdims=True)
p.shape

In [ ]:
# now get the prob of each row's correct label y
prob = p[torch.arange(embedding.shape[0]),Y]
prob.shape

In [ ]:
neg_log_like = - p[torch.arange(embedding.shape[0]),Y].log().mean()
neg_log_like

In [ ]:
## ------------------------------------------------------------
block = 3
X,Y = [] , []
for i in words:
    context = [0] * block
    for ch in i +'.':
        ix = str2int[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

X.shape,Y.shape

In [ ]:
L = torch.randn((27,2),requires_grad = True)
w1 = torch.randn((6,100),requires_grad = True)  # 6 inputs and 100 neurons in the first layer
b1 = torch.randn((100),requires_grad = True)  # biases for the 100 neurons
w2 = torch.randn((100,27),requires_grad = True)  # 100 inputs and 27 neurons in the first layer
b2 = torch.randn((27,),requires_grad = True) 
params = [L,w1,b1,w2,b2]

In [ ]:
'''
lre = torch.linspace(-3,0,1000) # 10^-3 to 10^0
lrs = 10**lre

lri = []
lossi = []
for i in range(1000):
    # mini batch
    rix = torch.randint(0,X.shape[0],(32,)) # 32 batch size
    
    # forward
    emb = L[X[rix]] # (32,3,2)
    layerone = torch.tanh(emb.view(emb.shape[0],6) @ w1 + b1) # (32,100)
    logits = layerone @ w2 + b2 # (32, 27)
    # counts = logits.exp()
    # p = counts / counts.sum(1,keepdim=True)
    # loss = - p[torch.arange(emb.shape[0]),Y].log().mean()
    loss = F.cross_entropy(logits,Y[rix])  # replace the three lines efficiently
    # print(loss.item())

    # backward
    for p in params:
        p.grad = None
    loss.backward()
    
    #update
    lr = lrs[i] # decayed learning rate
    for p in params:
        p.data += -lr * p.grad

    # track stats
    lri.append(lre[i])
    lossi.append(loss.item())

print(loss.item())

plt.plot(lri,lossi)

'''

sum(p.nelement() for p in params)

In [ ]:
# plt.plot(lri,lossi)

In [ ]:
for i in range(10000):
    # mini batch
    rix = torch.randint(0,X.shape[0],(32,)) # 32 batch size
    
    # forward
    emb = L[X[rix]] # (32,3,2)
    layerone = torch.tanh(emb.view(emb.shape[0],6) @ w1 + b1) # (32,100)
    logits = layerone @ w2 + b2 # (32, 27)
    loss = F.cross_entropy(logits,Y[rix])  # replace the three lines efficiently

    # backward
    for p in params:
        p.grad = None
    loss.backward()
    
    #update
    lr = 0.1  # a good learning rate found with the loss graph
    for p in params:
        p.data += -lr * p.grad

print(loss.item())

In [ ]:
# loss on entire dataset
emb = L[X] 
layerone = torch.tanh(emb.view(emb.shape[0],6) @ w1 + b1) 
logits = layerone @ w2 + b2 
loss = F.cross_entropy(logits,Y) 
loss

In [ ]:
# train valid test split => 80% 10% 10%
def build_dataset(words):
    block = 3
    X,Y = [] , []
    for i in words:
        context = [0] * block
        for ch in i +'.':
            ix = str2int[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    
    X = torch.tensor(X, device=device)
    Y = torch.tensor(Y, device=device)
    
    print(X.shape,Y.shape)
    return X,Y

import random
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtrn,Ytrn = build_dataset(words[:n1])
Xval,Yval = build_dataset(words[n1:n2])
Xtest,Ytest = build_dataset(words[n2:])

In [ ]:
L = torch.randn((27,10),requires_grad = True)   # 10 dimensional embedding vectors
w1 = torch.randn((30,200),requires_grad = True)  # 3x10 30 inputs and 300 neurons in the first layer
b1 = torch.randn((200),requires_grad = True)  # biases for the 300 neurons
w2 = torch.randn((200,27),requires_grad = True)  # 300 inputs and 27 neurons in the first layer
b2 = torch.randn((27,),requires_grad = True) 
params = [L,w1,b1,w2,b2]

In [ ]:
stepi=[]
lossi=[]
for i in range(30000):
    # mini batch
    rix = torch.randint(0,Xtrn.shape[0],(32,)) # 32 batch size
    
    # forward
    emb = L[Xtrn[rix]] # (32,3,2)
    layerone = torch.tanh(emb.view(emb.shape[0],w1.shape[0]) @ w1 + b1) # (32,100)
    logits = layerone @ w2 + b2 # (32, 27)
    loss = F.cross_entropy(logits,Ytrn[rix])  # replace the three lines efficiently

    # backward
    for p in params:
        p.grad = None
    loss.backward()
    
    #update
    lr = 0.01  # a good learning rate found with the loss graph
    for p in params:
        p.data += -lr * p.grad

    # track
    stepi.append(i)
    lossi.append(loss.item())

print(loss.item())

In [ ]:
plt.plot(stepi,lossi)

In [ ]:
## train loss
emb = L[Xtrn] 
layerone = torch.tanh(emb.view(emb.shape[0],w1.shape[0]) @ w1 + b1) 
logits = layerone @ w2 + b2 
loss = F.cross_entropy(logits,Ytrn) 
loss

In [ ]:
## validation
emb = L[Xval] 
layerone = torch.tanh(emb.view(emb.shape[0],w1.shape[0]) @ w1 + b1) 
logits = layerone @ w2 + b2 
loss = F.cross_entropy(logits,Yval) 
loss

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def reg(params):
    sum = 0
    for p in params:
        sum += (p**2).mean()
    sum /= len(params)
    return sum
        

In [ ]:
## trying to reduce loss

random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtrn,Ytrn = build_dataset(words[:n1])
Xval,Yval = build_dataset(words[n1:n2])
Xtest,Ytest = build_dataset(words[n2:])

batch = 32
neurons = 200
embed_size = 40
block = 3

L = torch.randn((27,embed_size),requires_grad = True, device=device)   
w1 = torch.randn((embed_size*block,neurons),requires_grad = True, device=device)  
b1 = torch.randn((neurons),requires_grad = True, device=device)  
w2 = torch.randn((neurons,27),requires_grad = True, device=device)  
b2 = torch.randn((27,),requires_grad = True, device=device) 
params = [L,w1,b1,w2,b2]

print(f"\nembedding size : {L.shape[1]} \nneurons: {w1.shape[1]} \nbatch size: {batch}")

stepi=[]
lossi=[]
for i in range(200000):
    
    # mini batch
    rix = torch.randint(0,Xtrn.shape[0],(batch,)) 
    
    # forward
    emb = L[Xtrn[rix]] 
    layerone = torch.tanh(emb.view(emb.shape[0],w1.shape[0]) @ w1 + b1) 
    logits = layerone @ w2 + b2 
    loss = F.cross_entropy(logits,Ytrn[rix]) + 0.005 * reg(params)

    # backward
    for p in params:
        p.grad = None
    loss.backward()
    
    #update
    lr = 0.1
    if i>60000:lr=0.07
    if i>120000:lr = 0.01
        
    for p in params:
        p.data += -lr * p.grad

    # track
    stepi.append(i)
    lossi.append(loss.item())


## train loss
emb = L[Xtrn] 
layerone = torch.tanh(emb.view(emb.shape[0],w1.shape[0]) @ w1 + b1) 
logits = layerone @ w2 + b2
loss = F.cross_entropy(logits,Ytrn)
print(f"training loss: {loss.item():.4f}")

## validation
emb = L[Xval] 
layerone = torch.tanh(emb.view(emb.shape[0],w1.shape[0]) @ w1 + b1) 
logits = layerone @ w2 + b2 
loss = F.cross_entropy(logits,Yval) 
print(f"Validation loss: {loss.item():.4f}")

In [ ]:
## observations
plt.plot(stepi,lossi)

In [ ]:
emb = L[Xtest] 
layerone = torch.tanh(emb.view(emb.shape[0],w1.shape[0]) @ w1 + b1) 
logits = layerone @ w2 + b2
loss = F.cross_entropy(logits,Ytest)
print(f"test loss: {loss.item():.4f}")